In [7]:
%matplotlib notebook 
import argparse
import fnmatch
import os
import shutil
import subprocess
# import h5py
import imageio
import matplotlib as mpl
# mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
# from imutils import face_utils
import pickle
import cv2
import face_alignment
import librosa
# import utils 
# from joblib import Parallel, delayed
from tqdm import tqdm
import dlib
import matplotlib.animation as manimation

# from torch import multiprocessing

  

In [8]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False,  device='cpu')


In [4]:

def get3DLmarks(frame_list, v_path):
    frame_num = len(frame_list)
    lmarks = np.zeros((frame_num, 68,3))
    for i in range(frame_num):
        lmark = fa.get_landmarks(frame_list[i])
#         print (lmark.shape)
        # lmark = FAN.get3Dfrom2D(i)
        if lmark is not None:
            landmark =  lmark[0]
        else:
            landmark = -np.ones((68, 3))
#         fig = plt.figure(figsize=plt.figaspect(.5))
#         tmp = v_path.split('/')
#         frame_name = os.path.join(root , 'img', tmp[-3], tmp[-2], tmp[-1][:-4] ,'%05d.png'%i)    
#         input = io.imread(frame_name)
#         preds = landmark
#         ax = fig.add_subplot(1, 2, 1)
#         ax.imshow(input)
#         ax.plot(preds[0:17,0],preds[0:17,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[17:22,0],preds[17:22,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[22:27,0],preds[22:27,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[27:31,0],preds[27:31,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[31:36,0],preds[31:36,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[36:42,0],preds[36:42,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[42:48,0],preds[42:48,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[48:60,0],preds[48:60,1],marker='o',markersize=6,linestyle='-',color='w',lw=2)
#         ax.plot(preds[60:68,0],preds[60:68,1],marker='o',markersize=6,linestyle='-',color='w',lw=2) 
#         ax.axis('off')

#         ax = fig.add_subplot(1, 2, 2, projection='3d')
#         surf = ax.scatter(preds[:,0]*1.2,preds[:,1],preds[:,2],c="cyan", alpha=1.0, edgecolor='b')
#         ax.plot3D(preds[:17,0]*1.2,preds[:17,1], preds[:17,2], color='blue' )
#         ax.plot3D(preds[17:22,0]*1.2,preds[17:22,1],preds[17:22,2], color='blue')
#         ax.plot3D(preds[22:27,0]*1.2,preds[22:27,1],preds[22:27,2], color='blue')
#         ax.plot3D(preds[27:31,0]*1.2,preds[27:31,1],preds[27:31,2], color='blue')
#         ax.plot3D(preds[31:36,0]*1.2,preds[31:36,1],preds[31:36,2], color='blue')
#         ax.plot3D(preds[36:42,0]*1.2,preds[36:42,1],preds[36:42,2], color='blue')
#         ax.plot3D(preds[42:48,0]*1.2,preds[42:48,1],preds[42:48,2], color='blue')
#         ax.plot3D(preds[48:,0]*1.2,preds[48:,1],preds[48:,2], color='blue' )

#         ax.view_init(elev=90., azim=90.)
#         ax.set_xlim(ax.get_xlim()[::-1])
#         plt.show()
        lmarks[i] = landmark
    np.save(v_path[:-4] + '.npy', lmarks)


In [5]:
def get_txt(folder):
    root = '/data2/lchen63/voxceleb/'

    file_list = []
    txt_f = open( os.path.join(root,'txt', 'v_test.txt'), 'wb') 
    for root, dirnames, filenames in os.walk(folder):
        for filename in filenames:
            if filename.endswith(('.mpg', '.mov', '.mp4')):
                filepath = os.path.join(root, filename)
#                 print (filepath, filename)
                file_list.append(filepath)
    print (file_list[:10])
    for line in file_list:
        txt_f.writelines(line)
        txt_f.write('\n')
    txt_f.close()
    
def _video2img2lmark(v_path):
    root = '/data2/lchen63/voxceleb/'

    count = 0
    tmp = v_path.split('/')
    frame_list = []
    if not os.path.exists(os.path.join(root , 'img') ):
        os.mkdir(os.path.join(root , 'img'))
    if not os.path.exists(os.path.join(root , 'img', tmp[-3]) ):
        os.mkdir(os.path.join(root , 'img', tmp[-3]))
    if not os.path.exists(os.path.join(root , 'img', tmp[-3], tmp[-2]) ):
        os.mkdir(os.path.join(root , 'img', tmp[-3], tmp[-2]))
    if not os.path.exists(os.path.join(root , 'img', tmp[-3], tmp[-2], tmp[-1][:-4]) ):
        os.mkdir(os.path.join(root , 'img', tmp[-3], tmp[-2], tmp[-1][:-4]))

    cap  =  cv2.VideoCapture(v_path)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:            
            frame_name = os.path.join(root , 'img', tmp[-3], tmp[-2], tmp[-1][:-4] ,'%05d.png'%count)
            cv2.imwrite(frame_name,frame)
            frame_list.append(frame)
            count += 1
        else:
            break
    get3DLmarks(frame_list,v_path)
#     for i in range(len(frame_list)):
#     print ('++++')
    
#         lmarks = Parallel(n_jobs=-1)(delayed(get3DLmarks)(frame_list[i]))
    
#     np.save(v_path[:-4] + '.npy', lmarks)
#     offset = 10
#     xLim = [np.min(lmarks[:,:,0])-offset, np.max(lmarks[:,:,0])+offset]
#     yLim = [np.min(lmarks[:,:,1])-offset, np.max(lmarks[:,:,1])+offset]
#     zLim = [np.min(lmarks[:,:,2])-offset, np.max(lmarks[:,:,2])+offset]
#     print (v_path)
#     utils.write_video(lmarks, speech, 8000, fileFolder, v_path[:-4]+'_2D',  xLim, yLim, fps=29.97)
#     utils.write_video3D(lmarks, speech, 8000, fileFolder, v_path[:-4]+'_3D', xLim, yLim, zLim, fps=29.97)
cmt = 0
global cmt
_video2img2lmark('/data2/lchen63/voxceleb/unzip/test_video/id00017/7t6lfzvVaTM/00007.mp4')
def video2img2lmark(list):
    length = len(list)
    for p in list:
        print ('{}/{}'.format(cmt, length))
        _video2img2lmark(p[:-1])
        cmt += 1
def video_transfer(txt):
    txt_f = open(txt, 'rb')
    list = txt_f.readlines()
    print (len(list))
    batch_size = int(len(list)/60)
    for i in range(60):
        process = multiprocessing.Process(target=video2img2lmark, args=(list[i*batch_size:(i+ 1) * batch_size],))
        process.start()
    process = multiprocessing.Process(target=video2img2lmark, args=(list[59*batch_size:],))
    process.start()

        
#     for line in txt_f.readlines():
#         v_path = line[:-1]
#         process = multiprocessing.Process(target=_video2img2lmark, args=(v_path,))
#         process.start()
#         _video2img2lmark(v_path)
# get_txt('/data2/lchen63/voxceleb/unzip/test_v')

# video_transfer('/data2/lchen63/voxceleb/txt/v_test.txt')




